In [2]:
import pandas as pd
import numpy as np
import re
import os

## 데이터 정제 규칙

### 1. 컬럼에 Q, A가 있으면 그대로 활용
### 2. 카카오톡처럼 1, 2, 3처럼 데이터가 합쳐져 있다면 패턴 파악 후 1, 2만 쓰기 or 1, 2, 3번 구분하기
##### (웰니스 데이터는 추가적인 방법 서치 필요)
### 3. 모든 데이터 합치기

### 1. 주제별 일상 대화 데이터

In [3]:
total_data = pd.read_csv('../Data/total_data.csv', encoding='utf-8', index_col=0)
sns_data = total_data[137078:]
sns_data.to_csv('../Data/sns_data.csv', encoding='utf-8')

In [4]:
# Question = sns_data[sns_data['group'] == '1']
# # Answer = sns_data[sns_data['group'] == '2']

# Question.reset_index(inplace=True, drop=True)
# Answer.reset_index(inplace=True, drop=True)

In [92]:
sns_data = sns_data['text']
sns_data.to_csv('../Data/sns_data.csv', encoding='utf-8')

In [93]:
sns_data

137078             엄마랑 단둘이 여행가기로 했어
137079                어머님이 좋아하시겠는걸?
137080          신세계백화점에서 쇼핑이 하고싶으시데
137081              그렇구나 어떤거 쇼핑하신대?
137082                  그냥 아이쇼핑 하신데
                     ...           
1756996          티가 나니까 눈치가 보이는 거죠!
1756997               훔쳐보는 거 티나나봐요.
1756998                      설렜겠어요.
1756999    잘 헤어질 수 있는 사이 여부인 거 같아요.
1757000          도피성 결혼은 하지 않길 바라요.
Name: text, Length: 1619923, dtype: object

### 2. 감성대화 말뭉치

In [11]:
gamsung = pd.read_excel('../Data/감성대화말뭉치(최종데이터)_Training.xlsx', index_col=0)

In [64]:
# gamsung_Q = gamsung['사람문장1']
# gamsung_Q = pd.concat([gamsung_Q, gamsung['사람문장2']])
gamsung2 = gamsung[['사람문장2', '시스템문장2']]
gamsung1 = gamsung[['사람문장1', '시스템문장1']]

In [67]:
gamsung2.columns = [['Q', 'A']]
gamsung1.columns = [['Q', 'A']]

In [72]:
gamsung_df = pd.concat([gamsung1, gamsung2])
gamsung_df.reset_index(drop=True, inplace=True)

In [110]:
gamsung_df.to_csv('../Data/gamsung.csv', encoding='utf-8', index=0)

### 3. 일반상식 데이터

In [75]:
common = pd.read_csv('../Data/common_data.csv', encoding='utf-8')

In [77]:
common.columns = [['Q', 'A']]

In [95]:
common.to_csv('../Data/common_data.csv', encoding='utf-8')

### 4. 네이버 리뷰 데이터

In [96]:
naver = pd.read_csv('../Data/naver_review.csv', encoding='utf-8')

In [97]:
naver

,text,group
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
146178,인간이 문제지.. 소는 뭔죄인가..,0
146179,평점이 너무 낮아서...,1
146180,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
146181,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [98]:
from datasets import load_dataset

In [99]:
raw_datasets = load_dataset("squad")

Downloading: 5.06kB [00:00, 1.08MB/s]                   
Downloading: 2.19kB [00:00, 691kB/s]                  


Downloading: 30.3MB [00:16, 1.79MB/s]                            
Downloading: 4.85MB [00:00, 13.5MB/s]                           


Dataset squad downloaded and prepared to /Users/yechansmacbook/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7. Subsequent calls will reuse this data.


In [100]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [117]:

def prepare_data(example):
    answer = example['A']['text'][0]
    example['answer_start'] = example['A']['answer_start'][0]
    example['answer_end'] = example['answer_start'] + len(answer)
    return example

In [103]:
gamsung_df = gamsung_df.map(prepare_data)

AttributeError: 'DataFrame' object has no attribute 'map'

In [111]:
dataset = load_dataset("csv", data_files = "../Data/gamsung.csv", split="train")

Using custom data configuration default


0 tables [00:00, ? tables/s]/Users/yechansmacbook/.cache/huggingface/modules/datasets_modules/datasets/csv/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2/csv.py:91: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  csv_file_reader = pd.read_csv(
/Users/yechansmacbook/.cache/huggingface/modules/datasets_modules/datasets/csv/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2/csv.py:91: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  csv_file_reader = pd.read_csv(
                                

Dataset csv downloaded and prepared to /Users/yechansmacbook/.cache/huggingface/datasets/csv/default-4c7f93b71128d18a/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


In [123]:
dataset[0]['A']

'많이 힘드시겠어요. 주위에 의논할 상대가 있나요?'

In [118]:
dataset = dataset.map(prepare_data, remove_columns=['answer'])

TypeError: string indices must be integers

In [121]:
dataset.filter(lambda x : len(x['A']['text']) != 1)

AttributeError: module 'dill._dill' has no attribute 'stack'

In [124]:
from transformers import AutoTokenizer

model_checkpoint = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 1.00k/1.00k [00:00<00:00, 436kB/s]
Downloading: 100%|██████████| 2.83M/2.83M [00:04<00:00, 689kB/s] 
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [126]:
context = dataset[0]["Q"]
question = dataset[0]["A"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'많이 힘드시겠어요. 주위에 의논할 상대가 있나요? 일은 왜 해도 해도 끝이 없을까? 화가 난다.'

In [127]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

많이 힘드시겠어요. 주위에 의논할 상대가 있나요? 일은 왜 해도 해도 끝이 없을까? 화가 난다.


In [128]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [129]:
inputs["overflow_to_sample_mapping"]

[0]

In [132]:
inputs = tokenizer(
    dataset[:10]["Q"],
    dataset[:10]["A"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 10 features.
Here is where each comes from: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].
